## Домашнее задание №1

### Предлагается реализовать поисковую систему, способную работать с proximity-оператором в запросе

#### Ограничения:
- В запросах встречаются только операторы OW/N и UW/N, где N - кол-во токенов, которое учитывается операторами
- Система должна уметь обрабатывать:
  - однотокенные запросы (без proximity-оператора), например: "молоко"
  - двухтокенные запросы с proximity-оператором между токенами, например: "молочный OW/1 ломтик"
- Система должна возвращать в ответе максимум 20 товаров
- Весь код должен быть абсолютно воспроизводимым - в режиме "Run all"

#### Цель - получить работающую поисковую систему с метриками качества на валидационном наборе запросов (прилагается к заданию):
- precision > 0.3
- recall > 0.4

#### Разбалловка за каждый пункт указана ниже, максимальное количество баллов за задание - 10

In [ ]:
from typing import Optional, Literal
import string
from dataclasses import dataclass

import pandas as pd
import numpy as np

from nltk.corpus import stopwords
from pymorphy3 import MorphAnalyzer

from tqdm import tqdm

np.random.seed(42)


### Дан корпус документов - база товаров с их именами

In [ ]:
dataset = pd.read_parquet("products_with_names.parquet")

In [ ]:
documents_dict = {
    doc[1]["product_id"]: doc[1]["name"] for doc in dataset.iterrows()
}

In [ ]:
@dataclass
class Document:
    doc_id: int
    name: str


documents = [Document(doc_id=doc[1]["product_id"], name=doc[1]["name"]) for doc in dataset.iterrows()]


### Пример реализации класса обработчика текста

Не обязательно использовать его в таком виде, можно реализовать любую обработку

In [ ]:
class TextProcessor:
    def __init__(self):
        self.symbols_to_replace = {"ё": "е"}
        self.stopwords = set(stopwords.words("russian"))
        self.linguist = MorphAnalyzer()

    def lowercase_text(self, text: str) -> str:
        # TODO: add some code here
        pass

    def replace_symbols(self, text: str) -> str:
        # TODO: add some code here
        pass

    def process_punctuation_simple(self, text: str) -> str:
        # TODO: add some code here
        pass

    def tokenize_simple(self, text: str) -> list[str]:
        # TODO: add some code here
        pass

    def remove_stopwords(self, doc: list[str]) -> list[str]:
        # TODO: add some code here
        pass

    def lemmatize_token(self, token: str) -> str:
        # TODO: add some code here
        pass
    
    def lemmatize_tokenized_text(self, tokenized_text: list[str]) -> list[str]:
        # TODO: add some code here
        pass

    def process_text(self, text: str) -> list[str]:
        text = self.lowercase_text(text)
        text = self.replace_symbols(text)
        text = self.process_punctuation_simple(text)
        text_tokens = self.tokenize_simple(text)
        text_tokens = self.remove_stopwords(text_tokens)
        return self.lemmatize_tokenized_text(text_tokens)
        

In [ ]:
text_processor = TextProcessor()

In [ ]:
@dataclass
class ProcessedDocument:
    doc_id: int
    name_tokens: list[str]


In [ ]:
documents_processed = [
    ProcessedDocument(
        doc_id=document.doc_id, 
        name_tokens=text_processor.process_text(document.name),
    ) 
    for document in tqdm(documents)
]

### (2 балла) Построение positional inverted index - обратного индекса с токенопозициями

In [ ]:
@dataclass
class DocIdWithTokenPositions:
    doc_id: int
    token_positions: list[int]


In [ ]:
def create_positional_inverted_index(
    documents_processed: list[ProcessedDocument],
    limit: Optional[int] = None,
) -> dict[str, list[DocIdWithTokenPositions]]:
    """
    Функция построения обратного индекса с токенопозициями

    Args:
        documents_processed (list[ProcessedDocument]): Набор документов корпуса с именами, прошедшими лингвистическую обработку
        limit (int, optional): Параметр для ограничения кол-ва документов для построения индекса

    Returns:
        dict[str, list[DocIdWithTokenPositions]]: hash-based обратный индекс с токенопозициями
    
    """
    
    # TODO: add some code here
    pass


In [ ]:
positional_inverted_index = create_positional_inverted_index(documents_processed)

### (3 балла) Реализация функции обработки запроса и слияния списков документов, учитывая позиции токенов в документах и proximity-оператор в запросе

In [ ]:
def merge_posting_lists_with_condition(
    posting_lists: tuple[DocIdWithTokenPositions, DocIdWithTokenPositions], 
    condition: Literal["OW/1", "OW/2", "OW/3", "UW/1","UW/2","UW/3"],
) -> list[int]:
    """
    Функция реализует слияние двух списков документов с условием proximity-оператора

    Args:
        posting_lists (tuple[DocIdWithTokenPositions, DocIdWithTokenPositions]): Кортеж из двух списков документов с токенопозициями
        condition (Literal["OW/1", "OW/2", "OW/3", "UW/1","UW/2","UW/3"]): Proximity-оператор

    Returns:
        list[int]: Результат слияния списков документов
    
    """
    
    # TODO: add some code here
    pass


In [ ]:
def search_over_positional_inverted_index(
    positional_inverted_index: dict[str, list[DocIdWithTokenPositions]], 
    documents_dict: dict[int, str],
    query: str,
    limit: Optional[int] = None,
) -> list[Document]:
    """
    Функция реализует поиск по обратному индексу

    Args:
        positional_inverted_index (dict[str, list[DocIdWithTokenPositions]]): Обратный индекс с токенопозициями
        documents_dict (dict[int, str]): Корпус документов
        query (str): Запрос пользователя
        limit (int, optional): Параметр, ограничивающий количество возвращаемых документов

    Returns:
        list[Document]: Результат поиска
    
    """
    
    # TODO: add some code here
    pass


In [ ]:
search_over_positional_inverted_index(positional_inverted_index, documents_dict, "картофель")[:2]

In [ ]:
search_over_positional_inverted_index(positional_inverted_index, documents_dict, "молочный OW/1 ломтик")[:2]

In [ ]:
search_over_positional_inverted_index(positional_inverted_index, documents_dict, "уксус UW/2 яблочный")[:2]

### Читаем валидационный набор запросов с позитивными примерами товаров

Метрики будут считаться на нем

In [ ]:
result_proximity_validation_query_positives = pd.read_parquet("result_proximity_validation_query_positives.parquet")

In [ ]:
result_proximity_validation_query_positives_dict = {
    row[1].query: row[1].products.tolist()
    for row in result_proximity_validation_query_positives.iterrows()
}

In [ ]:
ground_truths_list = []
search_results_list = []

In [ ]:
for query, ground_truth_products in result_proximity_validation_query_positives_dict.items():
    ground_truths_list.append(ground_truth_products)
    search_results_list.append(
        search_over_positional_inverted_index(positional_inverted_index, documents_dict, query, limit=20)
    )

### (3 балла - за соответствие требованиям к метрикам) Посчитаем метрики качества системы 

In [ ]:
@dataclass
class Metrics:
    precision: float
    recall: float
    f1_score: float
        
    def __repr__(self):
        return f"precision = {self.precision}\nrecall = {self.recall}\nf1_score = {self.f1_score}"


In [ ]:
def calculate_metrics(ground_truth_set: set[int], search_results_set: set[int]):
    
    tp = len(ground_truth_set.intersection(search_results_set))
    
    precision = tp / len(search_results_set) if len(search_results_set) > 0 else 0.0
    
    recall = tp / len(ground_truth_set) if len(ground_truth_set) > 0 else 0.0
    
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    return Metrics(precision=precision, recall=recall, f1_score=f1_score)


In [ ]:
def calculate_validation_metrics(ground_truth_products_lists, search_products_lists):
    metrics = []
    for ground_truth, search in zip(ground_truth_products_lists, search_products_lists):
        metrics.append(
            calculate_metrics(set(ground_truth), set(x.doc_id for x in search))
        )
    
    return Metrics(
        precision=np.mean([x.precision for x in metrics]),
        recall=np.mean([x.recall for x in metrics]),
        f1_score=np.mean([x.f1_score for x in metrics]),
    )
    

In [ ]:
calculate_validation_metrics(ground_truths_list, search_results_list)

### (2 балла) Сравнение метрик с поиском по обратному индексу без учета токенопозиций

Нужно:
- реализовать поиск над обратным индексом без токенопозиций;
- посчитать метрики качества (precision, recall) на том же валидационном наборе запросов (игнорируя proximity-операторы);
- сравнить метрики с первым подходом и сделать вывод о полезности токенопозиций


In [ ]:
# TODO: add some code here